<h3>Proceso</h3>
<p>Aquí nos encargamos de generar el dataframe con los promedios de los precios unitarios por barrio en el tiempo; y el dataFrame con los coeficientes</p>

In [ ]:
# No vamos a graficar, de momento
#%matplotlib inline

import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt

from machineLearning import dataFrameUtils
from machineLearning import processing
from machineLearning.descriptionExpansion import gClaves

In [ ]:
import time
start = time.time()

gDataFrameGlobal = dataFrameUtils.LeerCarpetaDataFrames('./processed/')
gBarriosPrecioUnitario = processing.CalcularPrecioUnitarioPromedioPorBarrio(gDataFrameGlobal)
gBarriosSuperfice = processing.CalcularSuperficiePromedioPorBarrio(gDataFrameGlobal)
gBarrios = processing.UnirPromediosPorBarrio(gBarriosPrecioUnitario, gBarriosSuperfice)
gBarrios.to_csv('PromediosBarrios.csv', encoding = 'utf-8', index = False)
gDataFrameGlobal = processing.AgregarPromediosPorBarrios(gDataFrameGlobal, gBarriosPrecioUnitario)
processing.AgregarCoeficientesGlobales(gDataFrameGlobal)
gCoeficientes = processing.ProcesoIterativo(gDataFrameGlobal, ['rooms', 'floor'])
gCoeficientes.to_csv('Coeficientes.csv', encoding = 'utf-8', index = False)

end = time.time()
totalTime = end - start
print('Time used: ' + str(totalTime))